In [ ]:
pip install mysql-connector-python pandas matplotlib seaborn 

In [ ]:

import pandas as pd
import mysql.connector
import matplotlib.pyplot as plt
import seaborn as sns
# --- connect to your mysql database---
conn = mysql.connector.connect(
    host ="localhost",
    user ="root",
    password="password",
    database="organic_farming_db"
    
)

# --- load data into pandas dataframes ---
agri_df = pd.read_sql("SELECT * FROM agriculture_projects;", conn)
carbon_df = pd.read_sql("SELECT * FROM carbon_monitoring;", conn)
fund_df = pd.read_sql("SELECT * FROM funding_data;", conn)


# ---merge datasets---
merged = (
    agri_df
    .merge(carbon_df, on ="project_id", how = "left")
    .merge(fund_df, on ="project_id", how ="left")
)
print ("data merged successfully. total records:", len(merged))
display(merged.head())

In [ ]:
# --- Example KPIS---
kpis = {
    "Total Projects": merged["project_id"].nunique(),
    "Verified Carbon Credits": merged["credits_verified"].sum(skipna=True),
    "Total Funding (USD)": merged["amount_USD"].sum(),
    "Average Soil Carbon Gain": (merged["soil_carbon_current"] - merged["soil_carbon_baseline"]).mean().round(3),   
}
print("\n--- Core KPIs ---")
for k, v in kpis.items():
    print(f"{k}:{v:}")


In [ ]:
# --- Visualization 1: Funding vs Credits by Region ---
plt.figure(figsize=(8,4))
sns.barplot(
    data=merged.groupby("region", as_index=False)
    .agg({"amount_usd": "sum", "credits_verified": "sum"}),
    x="region", y="amount_usd"
)
plt.title("Total Funding By Region (USD)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# --- Visualization 2: Carbon cerdit distribution ---
plt.figure(figsize=(8,4))
sns.histplot(merged["credits_verified"].dropna(), bins=20, kde=True)
plt.title("Distribution of Verified Carbon Credits")
plt.xlabel("Credits Verified")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

conn.close()